<h1> 어제 오른 내 주식, 과연 내일은? <h1>
    
## 학습 목표
    
---
    
- 시계열 데이터의 특성과 안정적(Stationary) 시계열의 개념을 이해한다.
- ARIMA 모델을 구성하는 AR, MA, Diffencing의 개념을 이해하고 간단한 시계열 데이터에 적용해 본다.
- 실제 주식 데이터에 ARIMA를 적용해서 예측 정확도를 확인해 본다.

## 목차
    
--- 
    
### 시계열 예측이란

- 미래를 예측한다는 것은 가능할까?
- Stationary한 시계열 데이터란?
- 시계열 데이터 사례분석
- Stationary 여부를 체크하는 통계적 방법

### ARIMA 시계열 예측

- 시계열 예측의 기본 아이디어 : Stationary하게 만들 방법은 없을까?
- ARIMA 모델의 개념
- ARIMA 모델 훈련과 추론

### 프로젝트 : 주식 예측에 도전해 보자
    
## 시계열 예측이란(1) 미래를 예측한다는 것은 가능할까?
    
미래 예측 시나리오를 생각해 봅시다.
    
- 지금까지의 주가변곡선을 바탕으로 다음 주가변동 예측
- 특정 지역의 기후데이터를 바탕으로 내일의 온도변화 예측
- 공장 센터데이터 변화이력을 토대로 이상 발생 예측
    
예측의 근거가 되는 시계열(Time-Series) 데이터가 있다는 것이다. 

시계열이란 시간 순서대로 발생한 데이터의 수열이라는 뜻이다. 굳이 간단한 수식의 형태를 빌리자면 아래와 같이 표현될 것이다.

Y = 
{
Y
t
:
t
∈
T
}
, where 
T
 is the index set
    
일정 시간 간격으로 발생한 데이터일 때가 많지만 꼭 그래야만 하는 것은 아니다. 
    
매일의 주식 거래 가격을 날짜-가격 형태로 날짜순으로 모아둔 데이터가 있다면 훌륭한 시계열 데이터가 되며, 이때 날짜가 인덱스(index) 역할을 하게 된다.

미래의 데이터를 예측하려 든다면 적어도 다음 두 가지의 전제가 필요

- 과거의 데이터에 일정한 패턴이 발견된다.
- 과거의 패턴은 미래에도 동일하게 반복될 것이다.

즉  안정적(Stationary) 데이터에 대해서만 미래 예측이 가능하다. - 안정적(Stationary)이다는 것은 시계열 데이터의 통계적 특성이 변하지 않는다는 뜻

시계열 데이터 분석은 완벽한 미래 예측을 보장하지는 않는데, 어떤 예측하지 못한 외부적 변수에 의해 시계열 데이터 분석의 전제가 되는 안정성(stationarity)이 훼손될 여지가 있기 때문이다다. 그럼에도 불구하고, 시계열 데이터 분석은 프로세스 내재적인 시간적 변화를 묘사하는데 아주 훌륭한 성능을 보인다.

## 시계열 예측이란(2) Stationary한 시계열 데이터

안정적인 시계열에서 시간의 추이와 관계없이 일정해야 하는 통계적 특성 세 가지는 평균, 분산, 공분산(정확히는 자기공분산-autocovariance이라고 해야 함) 이며 현실적으로 다루는 시계열 데이터는 이런 직관적인 안정성의 정의를 만족하는 정도면 충분히 ARIMA 등의 시계열 모델을 통해 분석이 용이하다다.

### 예시) 직전 5년 치 판매량 X(t-4), X(t-3), X(t-2), X(t-1), X(t)를 가지고 X(t+1)이 얼마일지 예측해보자.

이 예측이 의미가 있으려면 중요한 것은 t에 무관하게 이 예측이 맞아떨어져야 한다는 점이다.

t=2010일 때의 데이터를 가지고 X(2011)을 정확하게 예측하는 모델이라면 이 모델에 t=2020을 대입해도 이 모델이 X(2021)을 정확하게 예측할 수 있어야 한다는 것이다.

그러려면 t에 무관하게 X(t-4), X(t-3), X(t-2), X(t-1), X(t)의 평균과 분산이 일정 범위 안에 있어야 하하며,또한 X(t-h)와 X(t)는 t에 무관하게 h에 대해서만 달라지는 일정한 상관도를 가져야 한다.

그렇지 않으면 우리의 시계열 예측은 t에 따라 달라지는, 그때그때 달라요 예측이 되며, 이것은 과거의 패턴이 미래에도 반복될 것이다 라는 시계열 예측의 대전제를 무너뜨린다.

## 시계열 예측이란(3) 시계열 데이터 사례분석

데이터를 준비

$ mkdir -p ~/aiffel/EXPLORATION/14/stock_prediction

$ ln -s ~/data/ ~/aiffel/EXPLORATION/14/stock_prediction

### 시계열(Time Series) 생성

첫 번째로 다루어볼 데이터는 Daily Minimum Temperatures in Melbourne 이다. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import warnings
warnings.filterwarnings('ignore')

print('슝=3')

In [ ]:
dataset_filepath = os.getenv('HOME')+'/aiffel/stock_prediction/data/daily-min-temperatures.csv' 
df = pd.read_csv(dataset_filepath) 
print(type(df))
df.head()

시계열(Time Series)이란 것도 결국 시간 컬럼을 index로 하는 Series로 표현된다. 

우리가 읽어 들인 데이터 파일은 Pandas를 통해 2개의 컬럼을 가진 DataFrame으로 변환되었다.

DataFrame에서 Date 컬럼을 index로 삼아 시계열을 생성해 봅시다.

In [ ]:
# 이번에는 Date를 index_col로 지정해 주었습니다. 
df = pd.read_csv(dataset_filepath, index_col='Date', parse_dates=True)
print(type(df))
df.head()

시간 컬럼이 index가 되었지만 아직도 type은 DataFrame으로 나오는데, 우리가 찾는 시계열은 바로 여기 있다.

In [ ]:
ts1 = df['Temp']  # 우선은 데이터 확인용이니 time series 의 이니셜을 따서 'ts'라고 이름 붙여줍시다!
print(type(ts1))
ts1.head()

DataFrame인 df와 Series인 df['Temp']는 index 구조가 동일하므로 Numpy, Pandas, Matplotlib 등 많은 라이브러리들이 호환해서 지원해 주지만 그렇지 않은 경우도 간혹 발생하므로 여기서는 명확하게 Series 객체를 가지고 진행한다.

### 시계열 안정성의 정성적 분석

시계열이 준비되었으므로 시각화를 통해 안정성(Stationarity) 여부를 확인

In [ ]:
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 13, 6    # matlab 차트의 기본 크기를 13, 6으로 지정해 줍니다.

# 시계열(time series) 데이터를 차트로 그려 봅시다. 특별히 더 가공하지 않아도 잘 그려집니다.
plt.plot(ts1)

분석에 들어가기 전에 잊지 말아야 할 단계가 하나 있는데, 바로 결측치 유무 확인 이다.

In [ ]:
ts1[ts1.isna()]  # 시계열(Time Series)에서 결측치가 있는 부분만 Series로 출력합니다.

NaN이 들어있는 데이터는 통계분석 시에 많은 오류를 일으킬뿐더러 분석결과에도 악영향을 미치기 때문에 결측치가 있다면 조치가 필요하다.

만약 결측치가 있다면, 두가지 방법으로 결측치를 처리해 줄 수 있다.

- 결측치가 있는 데이터를 모두 삭제(drop)

- 결측치 양옆의 값들을 이용해서 적절히 보간(interpolate)하여 대입
ex) 2와 4 사이 데이터가 NaN이라면 이 값을 3으로 채우는 방식

이번에는 시간을 index로 가지는 시계열 데이터이므로 삭제하는 것보다는 보간하는 방법을 선택할것이다.

Pandas에서 결측치 보간을 처리하는 메소드는 다음과 같다. 

In [ ]:
# 결측치가 있다면 이를 보간합니다. 보간 기준은 time을 선택합니다. 
ts1=ts1.interpolate(method='time')

# 보간 이후 결측치(NaN) 유무를 다시 확인합니다.
print(ts1[ts1.isna()])

# 다시 그래프를 확인해봅시다!
plt.plot(ts1)

시간 추이에 따라 일정한 평균, 분산, 자기공분산의 패턴이 나타나는 것 같지만, 시계열 데이터의 통계적 특성을 좀 더 명료하게 시각화해 보자.

아래는 일정 시간 내 구간 통계치(Rolling Statistics)를 시각화해 보는 함수입니다.

현재 타임스텝 부터 window에 주어진 타임스텝 이전 사이

- 구간의 평균(rolling mean, 이동평균)과
- 표준편차(rolling std, 이동표준편차)를 원본 시계열과 함께 시각화해 보면 좀 더 뚜렷한 경향성을 확인할 수 있다.

In [ ]:
def plot_rolling_statistics(timeseries, window=12):
    
    rolmean = timeseries.rolling(window=window).mean()  # 이동평균 시계열
    rolstd = timeseries.rolling(window=window).std()    # 이동표준편차 시계열

     # 원본시계열, 이동평균, 이동표준편차를 plot으로 시각화해 본다.
    orig = plt.plot(timeseries, color='blue',label='Original')    
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)
    
print('슝=3')

In [ ]:
plot_rolling_statistics(ts1, window=12)

시각화한 결과를 보면 시계열 데이터가 안정적이라 볼수 있지만, 명확하게 단정지으려면 좀더 통계적인 접근이 필요할 것입니다. 

### 다른 데이터에 대해서도 비교해 보자.

이번에는 International airline passengers 데이터셋입니다. 월별 항공 승객 수(천명 단위)의 시계열 데이터인데 기온 변화 데이터셋과는 좀 다른 패턴이 나타나지 않을까 예상된다. 동일한 방법으로 분석해 보면

In [ ]:
dataset_filepath = os.getenv('HOME')+'/aiffel/stock_prediction/data/airline-passengers.csv' 
df = pd.read_csv(dataset_filepath, index_col='Month', parse_dates=True).fillna(0)  
print(type(df))
df.head()

In [ ]:
ts2 = df['Passengers']
plt.plot(ts2)

직관적으로 봐도 온도변화 데이터셋과는 다른 패턴이 보인다. 

시간의 추이에 따라 시계열의 평균과 분산이 지속적으로 커지는 패턴을 보입니다. rolling statistics를 추가해서 시각화해 보면..

In [ ]:
plot_rolling_statistics(ts2, window=12)

이렇게 시간의 추이에 따라 평균과 분산이 증가하는 패턴을 보인다면 이 시계열 데이터는 적어도 안정적이진 않다고 정성적인 결론을 내려볼 수 있을 것이다. 

그렇다면 이런 시계열 데이터에 대해서는 시계열 예측을 시도할 수 없는 것일까요? 

이후 스텝들에는 이런 불안정적(Non-Stationary) 시계열 데이터에 대한 시계열 분석 기법을 다루어 볼 것입니다.

위와 같이 우리는 시계열 데이터의 안정성을 시각화 방법을 통해 정성적으로 분석해 보았으며, 이것은 시계열 데이터를 다루는 가장 기본적인 접근법이라 할 수 있다.

하지만 시계열 데이터의 안정성을 평가하는 데는 보다 정량적인 방법이 있다. 